# Lab 10

## ReLU (with xavier initializer)

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
from time import gmtime, strftime
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])

W1 = tf.get_variable('weight1', [784, 512], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable('bias1', [512], initializer=tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable('weight2', [512, 512], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable('bias2', [512], initializer=tf.contrib.layers.xavier_initializer())
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)

W3 = tf.get_variable('weight3', [512, 512], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable('bias3', [512], initializer=tf.contrib.layers.xavier_initializer())
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)

W4 = tf.get_variable('weight4', [512, 512], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.get_variable('bias4', [512], initializer=tf.contrib.layers.xavier_initializer())
layer4 = tf.nn.relu(tf.matmul(layer3, W4) + b4)

W5 = tf.get_variable('weight5', [512, 10], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.get_variable('bias5', [10], initializer=tf.contrib.layers.xavier_initializer())
hypothesis = tf.nn.softmax(tf.matmul(layer4, W5) + b5)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

prediction = tf.argmax(hypothesis, axis=1)
is_correct = tf.equal(prediction, tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

epochs = 15
batch_size = 100
num_iteartions = int(mnist.train.images.shape[0] / batch_size)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)

summary = tf.summary.merge_all()

writer = tf.summary.FileWriter('./logs/lab-10/relu/%s' % (strftime("%Y_%b_%d_%H:%M:%S", gmtime())))
writer.add_graph(sess.graph)

for epoch in range(epochs):
    total_cost = 0

    for i in range(num_iteartions):
        batch_x_data, batch_y_data = mnist.train.next_batch(batch_size)

        s, cost_val, _ = sess.run([summary, cost, train], feed_dict={X: batch_x_data, Y: batch_y_data})

        total_cost += cost_val
        writer.add_summary(s, global_step=(epoch * num_iteartions + i))

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(total_cost / num_iteartions))

acc_val = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})
print("Accuracy: ", acc_val)

## ReLU (with xavier initializer) + Dropout

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from time import gmtime, strftime
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable('weight1', [784, 512], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable('bias1', [512])
_layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(_layer1, keep_prob=keep_prob)

W2 = tf.get_variable('weight2', [512, 512], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable('bias2', [512])
_layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(_layer2, keep_prob=keep_prob)

W3 = tf.get_variable('weight3', [512, 512], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable('bias3', [512])
_layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
layer3 = tf.nn.dropout(_layer3, keep_prob=keep_prob)

W4 = tf.get_variable('weight4', [512, 512], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.get_variable('bias4', [512])
_layer4 = tf.nn.relu(tf.matmul(layer3, W4) + b4)
layer4 = tf.nn.dropout(_layer4, keep_prob=keep_prob)

W5 = tf.get_variable('weight5', [512, 10], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.get_variable('bias5', [10])
hypothesis = tf.matmul(layer4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)

prediction = tf.argmax(hypothesis, axis=1)
is_correct = tf.equal(prediction, tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

epochs = 15
batch_size = 100
num_iteartions = int(mnist.train.images.shape[0] / batch_size)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)

summary = tf.summary.merge_all()

writer = tf.summary.FileWriter('./logs/lab-10/relu/%s' % (strftime("%Y_%b_%d_%H:%M:%S", gmtime())))
writer.add_graph(sess.graph)

for epoch in range(epochs):
    total_cost = 0

    for i in range(num_iteartions):
        batch_x_data, batch_y_data = mnist.train.next_batch(batch_size)

        s, cost_val, _ = sess.run([summary, cost, train], feed_dict={X: batch_x_data, Y: batch_y_data, keep_prob: 0.5})

        total_cost += cost_val
        writer.add_summary(s, global_step=(epoch * num_iteartions + i))

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(total_cost / num_iteartions))

acc_val = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1})
print("Accuracy: ", acc_val)

/home/jinhyuk/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost = 0.510115696
Epoch: 0002 cost = 0.223968711
Epoch: 0003 cost = 0.187260407
Epoch: 0004 cost = 0.160886810
Epoch: 0005 cost = 0.147330923
Epoch: 0006 cost = 0.136313333
Epoch: 0007 cost = 0.123007771
Epoch: 0008 cost = 0.119440071
Epoch: 0009 cost = 0.113938984
Epoch: 0010 cost = 0.107471216
Epoch: 0011 cost = 0.102643029
Epoch: 0012 cost = 0.104564836
Epoch: 0013 cost = 0.099155289
Epoch: 0014 cost = 0.095349218
Epoch: 0015 cost = 0.093514175
Accuracy:  0.9812


## Exercise - Batch Normalization

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
from time import gmtime, strftime
import tensorflow as tf

tf.reset_default_graph()

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])
is_training = tf.placeholder(tf.bool)

with tf.variable_scope('weight1', reuse=tf.AUTO_REUSE):
    W1 = tf.get_variable('weight1', [784, 512], initializer=tf.contrib.layers.xavier_initializer())
with tf.variable_scope('bias1', reuse=tf.AUTO_REUSE):
    b1 = tf.get_variable('bias1', [512])

layer1 = tf.matmul(X, W1) + b1
layer1 = tf.contrib.layers.batch_norm(layer1, center=True, scale=True, is_training=is_training)
layer1 = tf.nn.relu(layer1)

with tf.variable_scope('weight2', reuse=tf.AUTO_REUSE):
    W2 = tf.get_variable('weight2', [512, 512], initializer=tf.contrib.layers.xavier_initializer())
with tf.variable_scope('bias2', reuse=tf.AUTO_REUSE):
    b2 = tf.get_variable('bias2', [512])

layer2 = tf.matmul(layer1, W2) + b2
layer2 = tf.contrib.layers.batch_norm(layer2, center=True, scale=True, is_training=is_training)
layer2 = tf.nn.relu(layer2)

with tf.variable_scope('weight3', reuse=tf.AUTO_REUSE):
    W3 = tf.get_variable('weight3', [512, 512], initializer=tf.contrib.layers.xavier_initializer())
with tf.variable_scope('bias3', reuse=tf.AUTO_REUSE):
    b3 = tf.get_variable('bias3', [512])

layer3 = tf.matmul(layer2, W3) + b3
layer3 = tf.contrib.layers.batch_norm(layer3, center=True, scale=True, is_training=is_training)
layer3 = tf.nn.relu(layer3)

with tf.variable_scope('weight4', reuse=tf.AUTO_REUSE):
    W4 = tf.get_variable('weight4', [512, 512], initializer=tf.contrib.layers.xavier_initializer())
with tf.variable_scope('bias4', reuse=tf.AUTO_REUSE):
    b4 = tf.get_variable('bias4', [512])

layer4 = tf.matmul(layer3, W4) + b4
layer4 = tf.contrib.layers.batch_norm(layer4, center=True, scale=True, is_training=is_training)
layer4 = tf.nn.relu(layer4)

with tf.variable_scope('weight5', reuse=tf.AUTO_REUSE):
    W5 = tf.get_variable('weight5', [512, 10], initializer=tf.contrib.layers.xavier_initializer())
with tf.variable_scope('bias5', reuse=tf.AUTO_REUSE):
    b5 = tf.get_variable('bias5', [10])

layer5 = tf.matmul(layer4, W5) + b5
layer5 = tf.contrib.layers.batch_norm(layer5, center=True, scale=True, is_training=is_training)

hypothesis = tf.nn.relu(layer5)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train = optimizer.minimize(cost)

prediction = tf.argmax(hypothesis, axis=1)
is_correct = tf.equal(prediction, tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

epochs = 15
batch_size = 100
num_iteartions = int(mnist.train.images.shape[0] / batch_size)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)

summary = tf.summary.merge_all()

writer = tf.summary.FileWriter('./logs/lab-10/batch_normalization/%s' % (strftime("%Y_%b_%d_%H:%M:%S", gmtime())))
writer.add_graph(sess.graph)

for epoch in range(epochs):
    total_cost = 0

    for i in range(num_iteartions):
        batch_x_data, batch_y_data = mnist.train.next_batch(batch_size)

        s, cost_val, _ = sess.run([summary, cost, train], feed_dict={X: batch_x_data, Y: batch_y_data, is_training: True})

        total_cost += cost_val
        writer.add_summary(s, global_step=(epoch * num_iteartions + i))

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(total_cost / num_iteartions))

acc_val = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, is_training: False})
print("Accuracy: ", acc_val)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 cost = 0.257181159
Epoch: 0002 cost = 0.119386140
Epoch: 0003 cost = 0.086130186
Epoch: 0004 cost = 0.066374078
Epoch: 0005 cost = 0.053661050
Epoch: 0006 cost = 0.050136692
Epoch: 0007 cost = 0.040774702
Epoch: 0008 cost = 0.037753604
Epoch: 0009 cost = 0.033226976
Epoch: 0010 cost = 0.029461903
Epoch: 0011 cost = 0.028500218
Epoch: 0012 cost = 0.025268326
Epoch: 0013 cost = 0.023799406
Epoch: 0014 cost = 0.019142293
Epoch: 0015 cost = 0.018783726
Accuracy:  0.9798
